In [139]:
import numpy as np
import netCDF4
from scipy import interpolate
from scipy.interpolate import griddata
import sys
import os
import yaml
import glob
from loguru import logger
from pyproj import Proj, transform
from shapely.geometry import Point
from typing import Tuple
import matplotlib.pyplot as plt
import pyproj

In [140]:
def transform_coords(x: float, y: float, in_epsg: str, out_epsg: str) -> Tuple[float, float]:
    """
    Transforms coordinates from one projection to another using pyproj.

    Args:
        x: The x-coordinate.
        y: The y-coordinate.
        in_epsg: The EPSG code of the input projection.
        out_epsg: The EPSG code of the output projection.

    Returns:
        A tuple of the transformed (x, y) coordinates.
    """
    source_crs = pyproj.CRS(in_epsg)
    destination_crs = pyproj.CRS(out_epsg)
    transformer = pyproj.Transformer.from_crs(source_crs, destination_crs, always_xy=True)
    return transformer.transform(x, y)

with open('/Users/rricker/GitLab/projects/chione/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# declare sensor and target variable options
sensor = config["options"]["sensor"]
target_variable = config["options"]["target_variable"]
hem = config["options"]["hemisphere"]
out_epsg = config["options"]["out_epsg"]

# declare stacking processing options
t_start = config['options']['proc_step_options']['stacking']['t_start']
t_window_length = config['options']['proc_step_options']['stacking']['t_window']
t_series_length = config['options']['proc_step_options']['stacking']['t_length']

# declare histogram options
hist_n_bins = config['options']['proc_step_options']['stacking']['hist']['n_bins']
hist_range = config['options']['proc_step_options']['stacking']['hist']['range'][
    "freeboard" if "freeboard" in target_variable else "thickness"
]

# define directories
geojson_dir = config['dir'][sensor]['geojson']
sit_dir = config['dir'][sensor]['level2']
sic_dir = config['dir']['auxiliary']['ice_conc'][config['options']['ice_conc_product']]
sid_dir = config['dir']['auxiliary']['ice_drift'][config['options']['ice_drift_product']]


In [163]:
import netCDF4 
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
import re
import datetime
import h5py
import os
from astropy.time import Time

class SeaIceThicknessProducts:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

        self.file_list = None
        self.file_dates = None
        self.target_files = None
        self.product = None

        self.function_map = {
            ('icesat2', 'total_freeboard'): self.atl10_to_gdf,
            ('icesat2', 'sea_ice_thickness'): self.is2sitdat4_to_gdf,
            ('cryosat2', 'sea_ice_freeboard'): self.cci_l2p_to_gdf,
            ('cryosat2', 'sea_ice_thickness'): self.cci_l2p_to_gdf,
            ('envisat', 'sea_ice_freeboard'): self.cci_l2p_to_gdf,
            ('envisat', 'sea_ice_thickness'): self.cci_l2p_to_gdf
        }

        self.config = {
            'icesat2': {
                'total_freeboard': {
                    'hem_nh': '-01',
                    'hem_sh': '-02',
                    'date_str': '{14}',
                    'date_pt': '%Y%m%d%H%M%S'
                },
                'sea_ice_thickness': {
                    'hem_nh': 'IS2SITDAT4_01',
                    'hem_sh': 'IS2SITDAT4_02',
                    'date_str': '{14}',
                    'date_pt': '%Y%m%d%H%M%S'
                }
            },
            'cryosat2': {
                'sea_ice_thickness': {
                    'hem_nh': '-nh-',
                    'hem_sh': '-sh-',
                    'date_str': '{8}',
                    'date_pt': '%Y%m%d'
                },
                'sea_ice_freeboard': {
                    'hem_nh': '-nh-',
                    'hem_sh': '-sh-',
                    'date_str': '{8}',
                    'date_pt': '%Y%m%d'
                }
            },
            'envisat': {
                'sea_ice_thickness': {
                    'hem_nh': '-NH-',
                    'hem_sh': '-SH-',
                    'date_str': '{8}',
                    'date_pt': '%Y%m%d'
                },
                'sea_ice_freeboard': {
                    'hem_nh': '-NH-',
                    'hem_sh': '-SH-',
                    'date_str': '{8}',
                    'date_pt': '%Y%m%d'
                }
            }
        }

    def get_product(self):
        key = (self.sensor, self.target_variable)
        product = self.function_map[key]()
        self.product = product

    def read_atl10(self, filename, attributes=False):
        with h5py.File(filename, 'r') as fileid:
            atl10_data = {}
            atl10_attrs = {}
            atl10_beams = []
            for gtx in [k for k in fileid.keys() if bool(re.match(r'gt\d[lr]', k))]:
                try:
                    fileid[gtx]['freeboard_beam_segment']['beam_freeboard']['height_segment_id']
                except keyerror:
                    pass
                else:
                    atl10_beams.append(gtx)

            for gtx in atl10_beams:
                atl10_data[gtx] = {}
                atl10_data[gtx]['freeboard_beam_segment'] = {}
                atl10_data[gtx]['freeboard_beam_segment']['beam_freeboard'] = {}
                atl10_data[gtx]['freeboard_beam_segment']['geophysical'] = {}
                atl10_data[gtx]['freeboard_beam_segment']['height_segments'] = {}
                atl10_data[gtx]['leads'] = {}

                for key, val in fileid[gtx]['freeboard_beam_segment'].items():
                    if isinstance(val, h5py.Dataset):
                        atl10_data[gtx]['freeboard_beam_segment'][key] = val[:]
                    elif isinstance(val, h5py.Group):
                        for k, v in val.items():
                            atl10_data[gtx]['freeboard_beam_segment'][key][k] = v[:]

                if attributes:
                    # getting attributes of icesat-2 atl10 beam variables
                    atl10_attrs[gtx] = {}
                    atl10_attrs[gtx]['freeboard_beam_segment'] = {}
                    atl10_attrs[gtx]['freeboard_beam_segment']['beam_freeboard'] = {}
                    atl10_attrs[gtx]['freeboard_beam_segment']['geophysical'] = {}
                    atl10_attrs[gtx]['freeboard_beam_segment']['height_segments'] = {}
                    atl10_attrs[gtx]['leads'] = {}
                    # global group attributes for atl10 beam
                    for att_name, att_val in fileid[gtx].attrs.items():
                        atl10_attrs[gtx][att_name] = att_val
                    for key, val in fileid[gtx]['freeboard_beam_segment'].items():
                        atl10_attrs[gtx]['freeboard_beam_segment'][key] = {}
                        for att_name, att_val in val.attrs.items():
                            atl10_attrs[gtx]['freeboard_beam_segment'][key][att_name] = att_val
                        if isinstance(val, h5py.Group):
                            for k, v in val.items():
                                atl10_attrs[gtx]['freeboard_beam_segment'][key][k] = {}
                                for att_name, att_val in v.attrs.items():
                                    atl10_attrs[gtx]['freeboard_beam_segment'][key][k][att_name] = att_val

            # icesat-2 orbit_info group
            atl10_data['orbit_info'] = {}
            for key, val in fileid['orbit_info'].items():
                atl10_data['orbit_info'][key] = val[:]

            atl10_data['ancillary_data'] = {}
            atl10_attrs['ancillary_data'] = {}
            for key in ['atlas_sdp_gps_epoch']:
                # get each hdf5 variable
                atl10_data['ancillary_data'][key] = fileid['ancillary_data'][key][:]
                # getting attributes of group and included variables
                if attributes:
                    # -- variable attributes
                    atl10_attrs['ancillary_data'][key] = {}
                    for att_name, att_val in fileid['ancillary_data'][key].attrs.items():
                        atl10_attrs['ancillary_data'][key][att_name] = att_val

        return (atl10_data, atl10_attrs, atl10_beams)

    def atl10_to_gdf(self):
        gdf_list = list()
        for file in self.target_files:
            atl10_data, atl10_attrs, atl10_beams = self.read_atl10(file, attributes=True)
            beam_list = list()
            for beam in atl10_beams:
                tmp = pd.DataFrame.from_dict(atl10_data[beam]['freeboard_beam_segment']['beam_freeboard'])
                tmp['beam'] = beam
                tmp['beam_type'] = atl10_attrs[beam]['atlas_beam_type'].decode('utf8')
                beam_list.append(tmp)

            df = pd.concat([df for df in beam_list]).pipe(gpd.GeoDataFrame)
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
            gdf = gdf.to_crs(self.out_epsg)
            gdf = gdf[(gdf['beam_fb_height'] < 10.0) &
                      (gdf['latitude'] > 50.0)]
            gdf_list.append(gdf)

        gdf_final = pd.concat(gdf_list).pipe(gpd.GeoDataFrame)
        gdf_final.crs = gdf_list[0].crs

        gdf_final.rename(columns={"beam_fb_confidence": "total_freeboard_confidence",
                                  "beam_fb_height": "total_freeboard",
                                  "beam_fb_quality_flag": "total_freeboard_quality_flag",
                                  "beam_fb_sigma": "total_freeboard_sigma"}, inplace=True)

        return gdf_final.reset_index(drop=True)

    def is2sitdat4_to_gdf(self):
        gdf_list = list()
        for file in self.target_files:
            data = netCDF4.Dataset(file)

            start_idx = os.path.basename(file).find("bnum") + 5
            end_idx = start_idx + 4
            beam = os.path.basename(file)[start_idx:end_idx]

            d = {
                'latitude': np.array(data["latitude"]),
                'longitude': np.array(data["longitude"]),
                'sea_ice_freeboard': np.array(data["freeboard"]),
                'sea_ice_thickness': np.array(data["ice_thickness"]),
                'sea_ice_thickness_unc': np.array(data["ice_thickness_unc"]),
                'snow_depth': np.array(data["snow_depth"]),
                'time': Time(np.array(data["gps_seconds"]), format='gps').to_datetime(),
                'beam': beam,
                'beam_type': "strong"
            }
            df = pd.DataFrame(data=d)
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
            gdf = gdf.to_crs(self.out_epsg)
            gdf = gdf[gdf['latitude'] > 50.0]
            gdf_list.append(gdf)

        gdf_final = pd.concat(gdf_list).pipe(gpd.GeoDataFrame)
        gdf_final.crs = gdf_list[0].crs

        return gdf_final.reset_index(drop=True)

    def cci_l2p_to_gdf(self):
        data = netCDF4.Dataset(self.target_files[0])
        latitude_field = "lat" if "lat" in data.variables else "latitude"
        longitude_field = "lon" if "lon" in data.variables else "longitude"
        d = {
            'latitude': np.array(data[latitude_field]),
            'longitude': np.array(data[longitude_field]),
            'sea_ice_freeboard': np.array(data["sea_ice_freeboard"]),
            'sea_ice_thickness': np.array(data["sea_ice_thickness"]),
            'sea_ice_thickness_uncertainty': np.array(data["sea_ice_thickness_uncertainty"]),
            'snow_depth': np.array(data["snow_depth"]),
            'time': np.array(data["time"])
        }
        df = pd.DataFrame(data=d)
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=4326)
        return gdf.to_crs(self.out_epsg)

    def get_file_list(self, directory):
        hem = self.config[self.sensor][self.target_variable]['hem_' + self.hem]
        pattern = os.path.join(directory, "**", "*" + hem + "*")
        file_list = [file_path for file_path in glob.iglob(os.path.join(directory, "**", "*"), recursive=True)
                     if hem in os.path.basename(file_path.lower())]
        self.file_list = file_list

    def get_file_dates(self):
        config = self.config[self.sensor][self.target_variable]
        date_str = config['date_str']
        date_pt = config['date_pt']

        dates = [
            datetime.datetime.strptime(re.search(r'\d' + date_str, file).group(), date_pt)
            for file in self.file_list
        ]
        self.file_dates = dates

    def get_target_files(self, t0, t1):
        dates = self.file_dates
        file_list = self.file_list
        files = [file_list[dates.index(d)] for d in dates if d >= t0 and d < t1]
        self.target_files = files


In [164]:
i=10
d_sgn = 1
t0 = t_start + datetime.timedelta(days=i)
t1 = t_start + datetime.timedelta(days=i + 1)

sit_product = SeaIceThicknessProducts(hem=hem,sensor=sensor,target_variable=target_variable,out_epsg=out_epsg)
sit_product.get_file_list(sit_dir[target_variable])
sit_product.get_file_dates()
sit_product.get_target_files(t0,t1)
sit_product.get_product()


In [158]:
grid, cell_width = define_grid(
    parcel_grid_options["bounds"][0],
    parcel_grid_options["bounds"][1],
    parcel_grid_options["bounds"][2],
    parcel_grid_options["bounds"][3],
    432,
    config['options']['out_epsg'])

In [168]:
def grid_data(gdf, grid, var, var_str, hist_n_bins=None, hist_range=None, fill_nan=False, agg_mode=None):

    if agg_mode is None:
        agg_mode = ['mean', 'std']
    tmp_grid = grid.copy()
    merged = gpd.sjoin(gdf[var + ['geometry']].copy(), grid, how='left', op='within')
    if 'mean' in agg_mode: dissolve_mean = merged.dissolve(by='index_right', aggfunc=np.mean)
    if 'std' in agg_mode: dissolve_std = merged.dissolve(by='index_right', aggfunc=np.std)
    if 'sum' in agg_mode: dissolve_sum = merged.dissolve(by='index_right', aggfunc=np.sum)
    if 'cnt' in agg_mode: dissolve_cnt = merged.dissolve(by='index_right', aggfunc='count')
    if 'hist' in agg_mode:
        dissolve_hist = merged.dissolve(by='index_right', aggfunc=np.mean)
        dissolve_hist.reset_index(inplace=True)
        dissolve_hist[var[0]] = ''

    for i in range(0, len(var)):
        if 'mean' in agg_mode: tmp_grid.loc[dissolve_mean.index, var_str[i]] = dissolve_mean[var[i]].values
        if 'std' in agg_mode: tmp_grid.loc[dissolve_std.index, var_str[i] + '_std'] = dissolve_std[var[i]].values
        if 'sum' in agg_mode: tmp_grid.loc[dissolve_sum.index, var_str[i] + '_sum'] = dissolve_sum[var[i]].values
        if 'cnt' in agg_mode: tmp_grid.loc[dissolve_cnt.index, var_str[i] + '_cnt'] = dissolve_cnt[var[i]].values

        if 'hist' in agg_mode:
            for j in dissolve_hist.index_right.unique():
                tmp = merged[merged.index_right == j].drop(['index_right', 'geometry'], axis=1).reset_index(drop=True)
                tmp_hist = np.histogram(np.array(tmp[var[i]]),
                                        bins=hist_n_bins,
                                        range=(hist_range[0],hist_range[1]))[0]
                dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
                    map(str, tmp_hist))

            tmp_grid.loc[dissolve_hist.index_right, var_str[i] + '_hist'] = dissolve_hist[var[i]].values

    if not fill_nan:
        tmp_grid = tmp_grid.dropna()

    centroidseries = tmp_grid['geometry'].centroid
    tmp_grid['x'], tmp_grid['y'] = centroidseries.x, centroidseries.y
    tmp_grid = tmp_grid.set_index(['x', 'y'])
    return tmp_grid

In [206]:
sit = sit_product.product
target_variable= 'sea_ice_thickness'
target_variable_unc='sea_ice_thickness_uncertainty'
add_variable=['sea_ice_freeboard','snow_depth']
hist_n_bins = 40
hist_range = [0, 10]

tmp_grid = grid_data(sit, grid, [target_variable], [target_variable],
                     hist_n_bins, hist_range, agg_mode=['mean', 'std', 'hist'])
unc_grid = grid_data(sit, grid, [target_variable_unc], [target_variable_unc],
                     hist_n_bins, hist_range, agg_mode=['mean', 'cnt'])
add_grid = grid_data(sit, grid, add_variable, add_variable,
                     hist_n_bins, hist_range, agg_mode=['mean'])

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
/var/folders/sh/kwy4x62d2mv1cgnwsyd94gzr0000gn/T/ipykernel_24432/1895823185.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [202]:
tmp_grid[target_variable_unc] = unc_grid[target_variable_unc]/np.sqrt(unc_grid[target_variable_unc+'_cnt'])
tmp_grid[add_variable] = add_grid[add_variable]

In [207]:
add_grid[add_variable]

sea_ice_freeboard  snow_depth
x          y                                       
-1987500.0 -212500.0           0.017872    0.057039
           -187500.0           0.017782    0.057250
-1962500.0 -187500.0           0.028186    0.058941
-1937500.0 -187500.0          -0.006461    0.061407
-1912500.0 -187500.0           0.014221    0.063816
...                                 ...         ...
 837500.0   12500.0            0.093519    0.064886
            37500.0            0.004880    0.063083
 862500.0  -12500.0            0.083234    0.063639
            12500.0            0.066792    0.063624
            37500.0           -0.016760    0.060953

[1484 rows x 2 columns]

In [24]:
import netCDF4
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator
import datetime
import sys
import os
import re

class SeaIceConcentrationProducts:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

        self.file_list = None
        self.file_dates = None
        self.conc = 'here'

        self.function_map = {
            ('osi430b'): self.get_ice_concentration,
            ('osi450'): self.get_ice_concentration
        }

        self.config = {
            'osi430b': {
                'hem_nh': '_nh_',
                'hem_sh': '_sh_',
                'date_str': '{12}',
                'date_pt': '%Y%m%d%H%M',
                'date_offset': datetime.timedelta(days=0)
            },
            'osi450': {
                'hem_nh': '_nh_',
                'hem_sh': '_sh_',
                'date_str': '{12}',
                'date_pt': '%Y%m%d%H%M',
                'date_offset': datetime.timedelta(days=0)
            }
        }

    def get_ice_concentration(self,target_files):
        data = netCDF4.Dataset(target_files)
        x, y = transform_coords(np.ma.getdata(data.variables['lon'][:, :]).flatten(),
                                np.ma.getdata(data.variables['lat'][:, :]).flatten(),
                                'epsg:4326', self.out_epsg)

        value = np.ma.getdata(data.variables['ice_conc'][0, :, :]).flatten()
        xc, yc = np.meshgrid(np.ma.getdata(data.variables['xc'][:] * 1000.0),
                             np.ma.getdata(data.variables['yc'][:] * 1000.0))
        coords = np.transpose(np.vstack((x, y)))

        ice_conc = griddata(coords, value, (xc, yc), method='nearest')
        ice_conc[ice_conc < 15] = 0
        return {"xc": xc, "yc": yc, "ice_conc": ice_conc}

    def interp_ice_concentration(self, x, y):
        xc, yc = self.ice_conc[-1]["xc"][0, :], self.ice_conc[-1]["yc"][:, 0]
        ice_conc = self.ice_conc[-1]["ice_conc"]
        # Check if xc and yc are in descending order
        if xc[0] > xc[-1]:
            xc = xc[::-1]
            ice_conc = ice_conc[:, ::-1]
        if yc[0] > yc[-1]:
            yc = yc[::-1]
            ice_conc = ice_conc[::-1, :]

        interp_func = RegularGridInterpolator((xc, yc), ice_conc.T, method='linear')
        ice_conc_interp = interp_func((x,y))
        return ice_conc_interp.flatten() / 100.0

    def get_file_list(self, directory):
        config = self.config[self.product_id]
        hem = config['hem_' + self.hem]
        pattern = os.path.join(directory, "**", "*" + hem + "*")
        file_list = sorted(glob.glob(pattern, recursive=True))
        self.file_list = file_list

    def get_file_dates(self):
        config = self.config[self.product_id]
        date_str = config['date_str']
        date_pt = config['date_pt']
        dates = [
            datetime.datetime.strptime(re.search(r'\d' + date_str, file).group(), date_pt)
            for file in self.file_list
        ]
        self.file_dates = [date + config['date_offset'] for date in dates]

    def get_target_files(self, t0, t1):
        dt1d = datetime.timedelta(days=1)
        dates = self.file_dates
        file_list = self.file_list
        file = [file_list[dates.index(d)] for d in dates if d >= t0 and d < t1]
        if len(file) == 0:
            t0i, t1i = t0, t1
            while file and (abs(t0i-t0)<datetime.timedelta(days=10)) :
                file = (file_list[dates.index(d)] for d in dates if d >= t0i and d < t1i)
                t0i, t1i = t0i - dt1d, t1i - dt1d
        return file[0]
        

In [25]:
i=10
d_sgn = 1
dt1d = datetime.timedelta(days=1)
t0 = t_start + datetime.timedelta(days=i)
t1 = t_start + datetime.timedelta(days=i + 1)

sic_product = SeaIceConcentrationProducts(hem=hem,product_id=config['options']['ice_conc_product'],
                                          out_epsg=out_epsg)

sic_product.get_file_list(sic_dir)
sic_product.get_file_dates()

sic_product.target_files = sic_product.get_target_files(t0,t1)
sic_product.ice_conc = sic_product.get_ice_concentration(sic_product.target_files)

sic_product.target_files = sic_product.get_target_files(t0 + d_sgn * dt1d,t1 + d_sgn * dt1d)
sic_product.ice_conc_ahead = sic_product.get_ice_concentration(sic_product.target_files)


In [26]:
import netCDF4
import numpy as np
from scipy.interpolate import griddata
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
import datetime
import sys
import os
import re

class SeaIceDriftProducts(SeaIceConcentrationProducts):
    def __init__(self, **kwargs):
        
        super().__init__()
        
        for key, value in kwargs.items():
            setattr(self, key, value)

        self.ice_drift = None

        self.function_map = {
            ('osi405'): self.get_ice_drift,
            ('osi455'): self.get_ice_drift
        }

        self.config = {
            'osi405': {
                'hem_nh': '_nh_',
                'hem_sh': '_sh_',
                'date_str': '{12}',
                'date_pt': '%Y%m%d%H%M',
                'time_span': 48,
                'date_offset': datetime.timedelta(days=1)
            },
            'osi455': {
                'hem_nh': '_nh_',
                'hem_sh': '_sh_',
                'date_str': '{12}',
                'date_pt': '%Y%m%d%H%M',
                'time_span': 24,
                'date_offset': datetime.timedelta(days=0)
            }
        }

    def get_ice_drift(self, target_files, ice_conc):
        data = netCDF4.Dataset(self.target_files)
        unc = np.ma.getdata(data.variables['uncert_dX_and_dY'][0, :, :]).flatten()

        x0, y0 = transform_coords(np.ma.getdata(data.variables['lon'][:, :]).flatten()[unc != -1e10],
                                  np.ma.getdata(data.variables['lat'][:, :]).flatten()[unc != -1e10],
                                  'epsg:4326', self.out_epsg)

        x1, y1 = transform_coords(np.ma.getdata(data.variables['lon1'][:, :]).flatten()[unc != -1e10],
                                  np.ma.getdata(data.variables['lat1'][:, :]).flatten()[unc != -1e10],
                                  'epsg:4326', self.out_epsg)

        dx = x1 - x0
        dy = y1 - y0

        coords = np.transpose(np.vstack((x0, y0)))
        
        xc, yc = ice_conc["xc"], ice_conc["yc"]
        ice_conc = ice_conc["ice_conc"]
        
        dx_i = griddata(coords, dx, (xc, yc), method='nearest')
        dy_i = griddata(coords, dy, (xc, yc), method='nearest')

        dx_i[ice_conc == 0] = 0
        dy_i[ice_conc == 0] = 0

        self.ice_drift = {"xc": xc, "yc": yc, "dx": dx_i, "dy": dy_i}

    def drift_correction(self, x, y):
        time_span = self.config[self.product_id]['time_span']
        xc, yc = self.ice_drift["xc"][0, :], self.ice_drift["yc"][:, 0]
        dx, dy = self.ice_drift['dx'], self.ice_drift['dy']
        np.ma.getdata(dx)[np.ma.getdata(dx) == -1e+10] = 0
        np.ma.getdata(dy)[np.ma.getdata(dy) == -1e+10] = 0
        # Check if xc and yc are in descending order
        if xc[0] > xc[-1]:
            xc = xc[::-1]
            dx = dx[:, ::-1]
            dy = dy[:, ::-1]
        if yc[0] > yc[-1]:
            yc = yc[::-1]
            dx = dx[::-1, :]
            dy = dy[::-1, :]

        f_dx = RegularGridInterpolator((xc, yc), dx.T, method='linear')
        f_dy = RegularGridInterpolator((xc, yc), dy.T, method='linear')
        dx_proj = f_dx((x, y))
        dy_proj = f_dy((x, y))
        # output in m/h
        return dx_proj.flatten() / time_span, dy_proj.flatten() / time_span


In [144]:
import geopandas as gpd
import shapely as shp
import numpy as np
import sys
from shapely.geometry import Point

def interpolate_growth(data, interp_var, growth_range, grid, cell_width):
    # define parameters for growth interpolation
    eps = 1.8
    nbs = 260
    fsm = interpolate.interp1d(np.array([40.0, 90.0]), np.array([80, 10]))

    # perform linear fit
    data['fit'] = data.groupby('index_right').apply(lambda x: np.polyfit(x['dt_days'], x[interp_var], 1))
    data['growth'] = [coeff[0] for coeff in data.fit.values]
    data[(data['growth'] > growth_range[1]) | (data['growth'] < growth_range[0])] = np.nan

    data = data.drop(columns=['fit'])
    data.index.names = ['index']

    # gridding of growth coefficients
    data_grid = grid_data(data, grid, ['growth'], ['growth'], fill_nan=True)

    centroidseries = data_grid['geometry'].centroid
    data_grid['yc'], data_grid['xc'] = centroidseries.x, centroidseries.y

    # interpolation of growth for all valid target variable data points
    f_growth = interpolate.RBFInterpolator(np.vstack((np.array(data_grid.dropna()['yc']),
                                                      np.array(data_grid.dropna()['xc']))).transpose(),
                                           np.array(data_grid.dropna()['growth']),
                                           neighbors=nbs,
                                           smoothing=fsm(
                                               np.array(data_grid.dropna().geometry.centroid.to_crs(4326).geometry.y)),
                                           kernel='gaussian', epsilon=eps * 1 / cell_width)
    return f_growth, data_grid

def define_grid(xmin, ymin, xmax, ymax, n_cells, epsg):
    cell_size = (xmax - xmin) / n_cells
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax, cell_size):
        for y0 in np.arange(ymin, ymax, cell_size):
            # bounds
            x1 = x0 + cell_size
            y1 = y0 + cell_size
            grid_cells.append(shp.geometry.box(x0, y0, x1, y1))
    return gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=epsg), cell_size

def grid_data(gdf, grid, var, var_str, hist_n_bins=None, hist_range=None, fill_nan=False, agg_mode=None):

    if agg_mode is None:
        agg_mode = ['mean', 'std']
    tmp_grid = grid.copy()
    merged = gpd.sjoin(gdf[var + ['geometry']].copy(), grid, how='left', op='within')
    if 'mean' in agg_mode: dissolve_mean = merged.dissolve(by='index_right', aggfunc=np.mean)
    if 'std' in agg_mode: dissolve_std = merged.dissolve(by='index_right', aggfunc=np.std)
    if 'sum' in agg_mode: dissolve_sum = merged.dissolve(by='index_right', aggfunc=np.sum)
    if 'hist' in agg_mode:
        dissolve_hist = merged.dissolve(by='index_right', aggfunc=np.mean)
        dissolve_hist.reset_index(inplace=True)
        dissolve_hist[var[0]] = ''

    for i in range(0, len(var)):
        if 'mean' in agg_mode: tmp_grid.loc[dissolve_mean.index, var_str[i]] = dissolve_mean[var[i]].values
        if 'std' in agg_mode: tmp_grid.loc[dissolve_std.index, var_str[i] + '_std'] = dissolve_std[var[i]].values
        if 'sum' in agg_mode: tmp_grid.loc[dissolve_sum.index, var_str[i] + '_sum'] = dissolve_sum[var[i]].values

        if 'hist' in agg_mode:
            for j in dissolve_hist.index_right.unique():
                tmp = merged[merged.index_right == j].drop(['index_right', 'geometry'], axis=1).reset_index(drop=True)
                tmp_hist = np.histogram(np.array(tmp[var[i]]),
                                        bins=hist_n_bins,
                                        range=(hist_range[0],hist_range[1]))[0]
                dissolve_hist[var[i]][dissolve_hist.index[dissolve_hist.index_right == j][0]] = ' '.join(
                    map(str, tmp_hist))

            tmp_grid.loc[dissolve_hist.index_right, var_str[i] + '_hist'] = dissolve_hist[var[i]].values

    if not fill_nan:
        tmp_grid = tmp_grid.dropna()

    centroidseries = tmp_grid['geometry'].centroid
    tmp_grid['x'], tmp_grid['y'] = centroidseries.x, centroidseries.y
    tmp_grid = tmp_grid.set_index(['x', 'y'])
    return tmp_grid



grid, cell_width = define_grid(
    parcel_grid_options["bounds"][0],
    parcel_grid_options["bounds"][1],
    parcel_grid_options["bounds"][2],
    parcel_grid_options["bounds"][3],
    432,
    config['options']['out_epsg'])


In [215]:
np.mean(data['growth_interpolated'])

-0.001187064899141986

In [213]:
file = '/Volumes/data2/projects/chione/product/cryosat2/geojson_ex/sea_ice_thickness-cryosat2-nh-20191005-v100-100.geojson'
data = gpd.read_file(file)
data

,sea_ice_thickness,sea_ice_thickness_std,sea_ice_thickness_hist,sea_ice_thickness_uncertainty,sea_ice_freeboard,snow_depth,dt_days,ice_conc,sea_ice_thickness_corr,growth_interpolated,geometry
0,1.441155,0.730463,1 1 1 2 1 2 1 3 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.281434,0.136646,0.054009,0,0.231123,1.441155,-0.001193,MULTIPOINT (-1955000.00000 15000.00000)
1,0.941067,0.824807,4 0 1 0 1 3 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.323600,0.084329,0.053689,0,0.162975,0.941067,-0.001193,MULTIPOINT (-1955000.00000 25000.00000)
2,0.480897,0.875042,0 4 1 0 1 1 3 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.217529,0.002773,0.054795,0,0.261367,0.480897,-0.001193,MULTIPOINT (-1945000.00000 15000.00000)
3,-0.045719,0.093133,1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.496822,-0.019518,0.055367,0,0.289225,-0.045719,-0.001193,MULTIPOINT (-1935000.00000 15000.00000)
4,0.306781,0.228705,3 5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.277644,0.017114,0.056513,0,0.309925,0.306781,-0.001193,MULTIPOINT (-1925000.00000 15000.00000)
...,...,...,...,...,...,...,...,...,...,...,...
10943,0.478895,0.732134,0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.722223,0.028857,0.080297,-2,0.593650,0.476509,-0.001193,"MULTIPOINT (925000.00000 -285000.00000, 930689..."
10944,0.654325,0.416502,1 1 2 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.418712,0.047560,0.079090,-2,0.596267,0.651940,-0.001193,"MULTIPOINT (935000.00000 -285000.00000, 940689..."
10945,0.911024,0.282143,0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.737052,0.075207,0.076271,-2,0.459361,0.908639,-0.001193,"MULTIPOINT (965000.00000 -285000.00000, 970689..."
10946,1.009292,0.507030,0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0.747285,0.086044,0.074239,-2,0.376851,1.006907,-0.001193,"MULTIPOINT (975000.00000 -285000.00000, 980689..."


In [80]:
j=0

cday = start_date + datetime.timedelta(days=j)
subs = cday.strftime("%Y%m%d")
file_f = [i for i in list_f if subs in re.search('-(.+?)-*.geojson', os.path.basename(i)).group(1)]
file_r = [i for i in list_r if subs in re.search('-(.+?)-*.geojson', os.path.basename(i)).group(1)]
stack_f = gpd.read_file(file_f[0])
stack_r = gpd.read_file(file_r[0])
stack_r = stack_r[stack_r.dt_days != 0].reset_index(drop=True)
data = pd.concat([stack_f, stack_r], ignore_index=True)
data.crs=3413

In [114]:
#for j in range(t_series_length):

j=0

cday = start_date + datetime.timedelta(days=j)
subs = cday.strftime("%Y%m%d")
file_f = [i for i in list_f if subs in re.search('-(.+?)-*.geojson', os.path.basename(i)).group(1)]
file_r = [i for i in list_r if subs in re.search('-(.+?)-*.geojson', os.path.basename(i)).group(1)]
stack_f = gpd.read_file(file_f[0])
stack_r = gpd.read_file(file_r[0])
stack_r = stack_r[stack_r.dt_days != 0].reset_index(drop=True)
data = pd.concat([stack_f, stack_r], ignore_index=True)
data.crs=3413

print('here1')

# apply growth correction
traj_geom = data['geometry']
target_location = data["geometry"].apply(lambda g: g.geoms[-1])
data["geometry"] = target_location

print('here2')

merged = gpd.sjoin(data, grid, how='left', op='within')
geo = merged.groupby(['index_right', 'dt_days'], as_index=False).first()['geometry']
tmp = merged.groupby(['index_right', 'dt_days'], as_index=False).mean().assign(geometry=geo)

print('here3')

n_tiepoints = tmp.groupby('index_right')['dt_days'].count()
valid_indices = n_tiepoints[n_tiepoints >= min_n_tiepoints].index
tmp = tmp[tmp['index_right'].isin(valid_indices)]
tmp.set_index('index_right', inplace=True)

print('here4')

fun, _ = interpolate_growth(tmp, interp_var, growth_range, grid, cell_width)
growth_interpolated = fun(np.array([np.array(data.geometry.x), np.array(data.geometry.y)]).transpose())

print('here5')

data[interp_var + "_corr"] = growth_interpolated * (-data.dt_days.to_numpy()) + data[interp_var].to_numpy()
data["growth_interpolated"] = growth_interpolated
data["geometry"] = traj_geom

#data.to_file(os.path.join(outdir, os.path.basename(file_f[0]).replace("-f-", "-")), driver="GeoJSON")
#os.remove(file_f[0])
#os.remove(file_r[0])

here1
here2
here3
here4
here5
